In [1]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark

ss = SparkSession.builder.getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1579218525976_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import time
begin_time = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
data_preprocessed = ss.sql("select * from parquet.`/data`")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
train_df, valid_df = data_preprocessed.randomSplit([0.85, 0.15], seed=1)
train_df.cache()
valid_df.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[num_prev_so: int, mean_amp_prev_so: double, mean_duration_prev_so: double, amp_curr_so: double, duration_curr_so: int, curr_sleep_stage: int, time_elapsed_since_asleep: int, time_deep_sleep: int, time_light_sleep: int, time_rem_sleep: int, time_wake_sleep: int, eeg_min: double, eeg_max: double, eeg_mean: double, eeg_median: double, label: int]

In [5]:
sc.list_packages()

sc.install_pypi_package("h2o-pysparkling-2.4")

sc.install_pypi_package("h2o")

import h2o
h2o.init()

import pyspark
ss = pyspark.sql.SparkSession.builder.master('yarn')\
.config("spark.dynamicAllocation.enabled", True)\
.getOrCreate()

from pysparkling import *

import h2o
hc = H2OContext.getOrCreate(ss)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- -------
beautifulsoup4             4.8.1  
boto                       2.49.0 
jmespath                   0.9.4  
lxml                       4.4.1  
mysqlclient                1.4.4  
nltk                       3.4.5  
nose                       1.3.4  
numpy                      1.14.5 
pip                        19.3.1 
py-dateutil                2.2    
python36-sagemaker-pyspark 1.2.6  
pytz                       2019.3 
PyYAML                     3.11   
setuptools                 45.0.0 
six                        1.12.0 
soupsieve                  1.9.4  
wheel                      0.33.6 
windmill                   1.6    

Processing /var/lib/livy/.cache/pip/wheels/0d/2f/d2/7e0c36c53b57ca597430858ab3d72e1a612a10780016ff6765/h2o_pysparkling_2.4-3.28.0.1.post1-py2.py3-none-any.whl
Processing /var/lib/livy/.cache/pip/wheels/9c/9b/f4/eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e/tabulate-0.8.6-cp36-none-any.whl
Proc

In [6]:
hc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Sparkling Water Context:
 * Sparkling Water Version: 3.28.0.1-1-2.4
 * H2O name: sparkling-water-livy_application_1579218525976_0006
 * cluster size: 8
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (6,172.31.7.122,54321)
  (8,172.31.3.220,54321)
  (3,172.31.6.255,54321)
  (1,172.31.12.147,54321)
  (4,172.31.6.255,54323)
  (5,172.31.7.122,54323)
  (2,172.31.12.147,54323)
  (7,172.31.3.220,54323)
  ------------------------

  Open H2O Flow in browser: http://ip-172-31-13-63.us-west-1.compute.internal:54327 (CMD + click in Mac OSX)

    
 * Yarn App ID of Spark application: application_1579218525976_0006

In [7]:
train_h2o = hc.as_h2o_frame(train_df)
valid_h2o = hc.as_h2o_frame(valid_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
train_h2o['label'] = train_h2o['label'].asfactor()
valid_h2o['label'] = valid_h2o['label'].asfactor()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
train_h2o.summary()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         num_prev_so         mean_amp_prev_so    mean_duration_prev_so    amp_curr_so        duration_curr_so    curr_sleep_stage     time_elapsed_since_asleep    time_deep_sleep     time_light_sleep    time_rem_sleep      time_wake_sleep     eeg_min              eeg_max             eeg_mean              eeg_median         label
-------  ------------------  ------------------  -----------------------  -----------------  ------------------  -------------------  ---------------------------  ------------------  ------------------  ------------------  ------------------  -------------------  ------------------  --------------------  -----------------  -------
type     int                 real                real                     real               int                 int                  int                          int                 int                 int                 int                 real                 real                real                  real               enum
mins  

In [10]:
from h2o.estimators.xgboost import H2OXGBoostEstimator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
predictors = train_h2o.names[:]
response = "label"
predictors.remove(response)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
model_xgb = H2OXGBoostEstimator(nfolds=3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
model_xgb.train(x=predictors,
                y=response,
                training_frame=train_h2o,
                validation_frame=valid_h2o)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

xgboost Model Build progress: [###########################################] 100%

In [14]:
model_xgb

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_model_python_1579225752259_1


Model Summary: 
    number_of_trees
--  -----------------
    50


ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.36321601661888103
RMSE: 0.6026740550404348
LogLoss: 0.9523631059038783
Mean Per-Class Error: 0.5078580689713582

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
0       1      2      Error     Rate
------  -----  -----  --------  -----------------
77972   12536  6596   0.197026  19,132 / 97,104
31650   23841  11278  0.642933  42,928 / 66,769
24765   15247  18518  0.683615  40,012 / 58,530
134387  51624  36392  0.458951  102,072 / 222,403

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.541049
2    0.826293
3    1

ModelMetricsMultinomial: xgboost
** Reported on validation data. **

MSE: 0.3775884197612103
RMSE: 0.6144822371405135
LogLoss: 0.9884414105191188
Mean Per-Class Error: 0.542289270918076

Confusion Matrix: Row 

In [15]:
print(model_xgb.confusion_matrix(train_h2o))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
0       1      2      Error     Rate
------  -----  -----  --------  -----------------
77972   12536  6596   0.197026  19,132 / 97,104
31650   23841  11278  0.642933  42,928 / 66,769
24765   15247  18518  0.683615  40,012 / 58,530
134387  51624  36392  0.458951  102,072 / 222,403

In [16]:
elapsed = time.time()-begin_time
elapsed

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

301.8973639011383